In [1]:
# ------------------------ Install Required Packages ------------------------------

!pip install transformers torch sentencepiece pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# ---------------------- TRANSLATION MODEL (WHISPER) ----------------------

# Import required libraries
from google.colab import files
import pandas as pd
from transformers import pipeline
import os
import warnings

# Suppress syntax warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

# Function to upload a file using Google Colab's file uploader
def upload_file():
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f"Uploaded file: {filename}")
        return filename

# Function to read a text file (.txt)
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [f.read()]  # Return as a list for consistency

# Function to read an Excel file (.xlsx) and extract a specific column
def read_excel_file(file_path, column_name=None):
    df = pd.read_excel(file_path)
    print("\nExcel Sheet Preview:")
    print(df.head())

    # Prompt for column name if not provided
    if column_name is None:
        column_name = input("\n🛠 Enter the column name containing the text to translate: ").strip()

    # Validate column existence
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in Excel file.")

    return df[column_name].astype(str).tolist()

# Function to translate a given text using a Hugging Face translation model
def translate_text(text, source_lang="en", target_lang="hi"):
    model_name = f"Helsinki-NLP/opus-mt-{source_lang}-{target_lang}"
    translator = pipeline("translation", model=model_name)
    result = translator(text, max_length=1000)
    return result[0]['translation_text']

# Main execution function
def main():
    print("Please upload a .txt or .xlsx file containing the script...")
    file_path = upload_file()
    ext = os.path.splitext(file_path)[-1].lower()  # Get file extension

    # Read file based on its type
    if ext == '.txt':
        texts = read_text_file(file_path)
    elif ext == '.xlsx':
        texts = read_excel_file(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a .txt or .xlsx file.")

    source_lang = "en"  # Assume uploaded file is in English
    target_lang = input("\nEnter target language code (e.g., 'hi' for Hindi, 'fr' for French): ").strip()

    translated_texts = []
    print("\nTranslating texts...")

    # Translate each text
    for text in texts:
        translated = translate_text(text, source_lang, target_lang)
        translated_texts.append(translated)

    # Save translated output
    if ext == '.txt':
        # Save translated text into a new .txt file
        output_file = "VISPER Translated Script.txt"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(translated_texts[0])  # Only one big text block
    elif ext == '.xlsx':
        # Save original and translated texts into a new Excel file
        df_result = pd.DataFrame({
            "Original Text": texts,
            f"Translated Text ({target_lang})": translated_texts
        })
        output_file = "VISPER Translated Script.xlsx"
        df_result.to_excel(output_file, index=False)

    print(f"\nTranslation complete! Downloading '{output_file}'...")
    files.download(output_file)

# Execute main function when script is run
if __name__ == "__main__":
    main()
